## This notebook gives instructions on advanced scenarios

In [ ]:
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from IPython.display import clear_output
import time


In [ ]:
from ray_on_aml.core import Ray_On_AML

ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="d15-v2",additional_pip_packages=['torch==1.10.0', 'torchvision', 'sklearn'], maxnode=1)
ray = ray_on_aml.getRay()
# Note that by default, ci_is_head=True which means  compute instance as head node and all nodes in the remote compute cluster as workers 
# But if you want to use one of the nodes in the remote AML compute cluster is used as head node and the remaining are worker nodes.
# then simply specify ray = ray_on_aml.getRay(ci_is_head=False)
# To install additional library, use additional_pip_packages and additional_conda_packages parameters.
time.sleep(50)
ray.cluster_resources()

## Loading custom resources such as files and python modules to run interactively

In [ ]:
#Create a new client object
import ray
from files import utils 
client = ray.init(f"ray://{ray_on_aml.headnode_private_ip()}:10001", runtime_env={"py_modules": ['files']})


@ray.remote
class test():
    def test_method(self):
        return utils.Foo().bar()
        return(os.listdir())
test_obj = test.remote()
print(ray.get(test_obj.test_method.remote()))
client.disconnect() 
#disconnect after use, if you don't do this, you will have error if you run the client connection again


Learn more at https://docs.ray.io/en/latest/handling-dependencies.html